In [34]:
from cdo import *
cdo = Cdo()
import my_shell_tools
import os
import os.path
import CMIP5_functions as cmip5

# import functions from file
from GLENS_functions import *

In [33]:
%load_ext autoreload
%autoreload 2

# Define Variables

In [23]:
# model_version = "b.e15.B5505C5WCCML45BGCR"
# resolution = "f09_g16" # ~1 degree resolution
exp = "feedback" # or "control"
run = "001"
comp = "cam"
hist_file = "h0" # or "h3" ?
variable = "TREFHT" # or others
dates = "201001-203012"

# file_format = "{model_version}.{resolution}.{exp}.{run}.{comp}.{hist_file}.{variable}.{dates}.nc"

short_format = "{exp}.{run}.{comp}.{hist_file}.{variable}.{dates}.nc"

feedback_runs = [str(IDX+1).zfill(3) for IDX in xrange(21)]
control_runs = ['001','002','003','021']
# control short runs are all between 001-021 not included in control_runs
control_short_runs = [x for x in feedback_runs if x not in control_runs]

monthly_variables = ['LHFLX','PRECC','PRECL','PRECTMX','TREFHT','TREFHTMX']
daily_variables = ['LHFLX','PRECT','TREFHT','TREFHTMX']

feedback_dates = '202001-209912'
control_dates = '201001-209912'

raw_monthly_dir = "/n/home03/pjirvine/keithfs1_pji/GLENS/raw_monthly_data/"
temp_dir = "/n/home03/pjirvine/keithfs1_pji/GLENS/temp_merged/"
ex_nc = "feedback.021.cam.h0.TREFHTMX.209001-209912.nc"


In [108]:
"""
sbatch maker
"""
script_name = 'ann_mean_std'
sbatch_maker('python '+script_name+'.py', script_name+'.sbatch', more_commands='', JOB=script_name, PART='general', NODES='1', CORES='1', MEM='3000', TIME='0-00:30')
# SUBMIT WITH THIS LINE
print linux_out('sbatch '+script_name+'.sbatch')

Submitted batch job 17314466



# Daily data merging - needed?

In [37]:
%%time

"""
TEST DAILY MERGE SCRIPT - allow 4 minutes for set of files
"""

"""
Define variables which specify which files to process
"""

in_dir = '/n/home03/pjirvine/keithfs1_pji/GLENS/raw_daily_data/'
out_dir = '/n/home03/pjirvine/keithfs1_pji/GLENS/'

runs = [str(IDX+1).zfill(3) for IDX in xrange(21)]
variables = ['LHFLX','PRECC','PRECL','PRECTMX','TREFHT','TREFHTMX']

run_name = 'feedback.021.cam.h3.TREFHTMX.'
append = '202001-209912.nc'

"""
Main loop
"""

# for var in variables:
#     for run in runs:
        
#         run_name = seach_format.format(run=run, var=var)
years_out = merge_years(run_name, append, in_dir, out_dir)

print run_name, years_out

feedback.021.cam.h3.TREFHTMX. [u'2020 2021 2022 2023 2024 2025 2026 2027 2028 2029 2030 2031 2032 2033 2034 2035 2036 2037 2038 2039 2040 2041 2042 2043 2044 2045 2046 2047 2048 2049 2050 2051 2052 2053 2054 2055 2056 2057 2058 2059 2060 2061 2062 2063 2064 2065 2066 2067 2068 2069 2070 2071 2072 2073 2074 2075 2076 2077 2078 2079 2080 2081 2082 2083 2084 2085 2086 2087 2088 2089 2090 2091 2092 2093 2094 2095 2096 2097 2098 2099 2100']
CPU times: user 7.83 ms, sys: 7.05 ms, total: 14.9 ms
Wall time: 3min 27s


# OLD NorESM processing script

In [34]:
"""
This script does the basic monthly processing for the NorESM1-M data 
"""

data_dir = '/net/keithfs1/srv/export/keithfs1/share_root/pjirvine/NorESM/'

exp_list = ['rcp45', 'rcp85', 'G6cct',  'G6ss',  'G6sulf']
var_list = ['evspsbl', 'pr', 'tas']
run_list = ['r1i1p1','r2i1p1','r3i1p1']

dir_struct = '{exp}/mon/atmos/{var}/{run}/'
file_struct = '{var}_Amon_NorESM1-ME_{exp}_{run}_202001-210012.nc' 

out_dir = '/n/home03/pjirvine/projects/NorESM_comparison/summary_data/'
outfile_struct_base = '{var}_{exp}_{run}'

def process(exp, var, run):

    spec_dict = {'exp': exp, 'var': var, 'run': run}

    file_name = data_dir + dir_struct.format(**spec_dict) + file_struct.format(**spec_dict)

    # date_file = cdo.seldate(date_range, input=file_name, output='test.nc')
    years_start='2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039'
    years_end='2080,2081,2082,2083,2084,2085,2086,2087,2088,2089,2090,2091,2092,2093,2094,2095,2096,2097,2098,2099'

    def selyear_process(years, append):
        years_file = cdo.selyear(years, input=file_name)#, ourput='test.nc')
        cdo.timmean(input=years_file, output=out_dir + outfile_struct_base.format(**spec_dict)+append+'_mean.nc')
        cdo.timstd(input=years_file, output=out_dir + outfile_struct_base.format(**spec_dict)+append+'_std.nc')

    selyear_process(years_start, '_2020_2040')
    selyear_process(years_end, '_2080_2100')

for exp in exp_list:
    for var in var_list:
        for run in run_list:
            process(exp,var,run)    

# simple function to return linux output

In [5]:
def linux_out(command):
    import os
    # command = string as typed into linux terminal
    stream = os.popen(command)
    # return output
    return stream.read()

# Sbatch maker function

In [6]:
def sbatch_maker(script_command, sbatch_filename, more_commands='', JOB='Test', PART='general', NODES='1', CORES='1', MEM='100', TIME='0-00:10'):
    """
This function makes an SBATCH script called "sbatch_filename" which runs the script given in script_command.
The properties of the job will be set by the key words.
REMEMBER - This sbatch job will inherit
"""
    
    slurm_template = """#!/bin/bash
#
#SBATCH -J {JOB}
#SBATCH -p {PART}                # partition (queue)
#SBATCH -N {NODES}                      # number of nodes
#SBATCH -n {CORES}                      # number of cores
#SBATCH --mem={MEM}                 # memory pool for all cores
#SBATCH -t {TIME}                 # time (D-HH:MM)
#SBATCH -o /n/home03/pjirvine/slurm/out/SLURM.%j.%N.out        # STDOUT
#SBATCH -e /n/home03/pjirvine/slurm/err/SLURM.%j.%N.err        # STDERR
#SBATCH --mail-type=ALL
#                               Type of email notification- BEGIN,END,FAIL,ALL
#SBATCH --mail-user=p.j.irvine@gmail.com
#                               Email to which notifications will be sent

{MORE_COMMANDS}
{SCRIPT}

############################## slurm ends ##############################
"""

    # Fill in the slurm template using kwargs
    sbatch_text = slurm_template.format(SCRIPT=script_command, MORE_COMMANDS=more_commands, JOB=JOB, PART=PART, NODES=NODES, CORES=CORES, MEM=MEM, TIME=TIME)

    # Write to file given by sbatch_filename
    with open(sbatch_filename, "w") as text_file:
        text_file.write(sbatch_text)
    
    # Return contents of sbatch file
    return sbatch_text

#def sbatch_maker ENDS

# Example showing sbatch maker in use
This will run the python script "python_test.py" that we will write below

In [17]:
sbatch_text = sbatch_maker('python python_test.py', 'py_test.sbatch', MEM='1', TIME='0-00:01')

This writes out a simple python script. This python script just creates a file and writes to it.

In [16]:
%%writefile python_test.py
with open('python_ran.txt', "w") as text_file:
    text_file.write('it worked')

Writing python_test.py


This executes a shell command to run the sbatch script, submitting the job that will run the python script.

In [18]:
! sbatch py_test.sbatch

Submitted batch job 17112004
